<a href="https://colab.research.google.com/github/hsunsmile/ComfyUI_VLM_nodes/blob/main/nightly/comic_diffusion_v2_comfyui_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

!apt -y update -qq
!apt -y install -qq aria2 rsync

In [ ]:
!wget https://s3.amazonaws.com/mountpoint-s3-release/latest/x86_64/mount-s3.deb
!apt install -y ./mount-s3.deb

!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O /content/cloudflared-linux-amd64 && chmod 777 /content/cloudflared-linux-amd64

In [ ]:
!pip install -q protobuf==5.29.1 numpy==2.0.0 -U
!pip install -q torch==2.8.0 torchvision torchaudio torchtext torchdata --extra-index-url https://download.pytorch.org/whl/cu121 -U
!pip install -q xformers triton -U
!pip install -q mediapipe==0.10.21 addict yapf fvcore omegaconf av ftfy
!pip install -q torchdiffeq ftfy gguf sageattention nunchaku
!pip install -q color-matcher

In [ ]:
import torch

if torch.cuda.is_available():
    print("GPU detected, installing nunchaku...")
    !pip install https://github.com/nunchaku-tech/nunchaku/releases/download/v1.0.0/nunchaku-1.0.0+torch2.8-cp312-cp312-linux_x86_64.whl
else:
    print("No GPU detected, skipping nunchaku installation.")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# #@title Environment Setup

from pathlib import Path
import os
import sys
import tarfile

OPTIONS = {}

USE_GOOGLE_DRIVE = True  #@param {type:"boolean"}
UPDATE_COMFY_UI = True  #@param {type:"boolean"}
USE_COMFYUI_MANAGER = True  #@param {type:"boolean"}
INSTALL_CUSTOM_NODES_DEPENDENCIES = True  #@param {type:"boolean"}
OPTIONS['USE_GOOGLE_DRIVE'] = USE_GOOGLE_DRIVE
OPTIONS['UPDATE_COMFY_UI'] = UPDATE_COMFY_UI
OPTIONS['USE_COMFYUI_MANAGER'] = USE_COMFYUI_MANAGER
OPTIONS['INSTALL_CUSTOM_NODES_DEPENDENCIES'] = INSTALL_CUSTOM_NODES_DEPENDENCIES

current_dir = !pwd
LOCAL_WORKSPACE = f"{current_dir[0]}/ComfyUI_local"
DRIVE_WORKSPACE = "/content/drive/MyDrive/ComfyUI"
LOCAL_TAR_PATH = f"{current_dir[0]}/ComfyUI_local.tar.gz"
DRIVE_TAR_PATH = "/content/drive/MyDrive/ComfyUI_local.tar.gz"

os.environ['COMFYUI_PATH'] = LOCAL_WORKSPACE
sys.path.append(LOCAL_WORKSPACE)

if OPTIONS['USE_GOOGLE_DRIVE']:
    !echo "Mounting Google Drive..."
    %cd /

    from google.colab import drive
    drive.mount('/content/drive')

    !echo "Attempting to extract ComfyUI from Google Drive archive..."
    if os.path.exists(DRIVE_TAR_PATH):
        !echo "Copying archive from Google Drive..."
        !cp "{DRIVE_TAR_PATH}" "{LOCAL_TAR_PATH}"
        try:
            !echo "Extracting archive..."
            with tarfile.open(LOCAL_TAR_PATH, 'r:gz') as tar:
                tar.extractall(path=current_dir[0])
            !echo "Extraction complete. Using extracted files."

            # Fix nested models directory after extraction
            nested_models_path = os.path.join(LOCAL_WORKSPACE, "models", "models")
            correct_models_path = os.path.join(LOCAL_WORKSPACE, "models")
            if os.path.exists(nested_models_path):
                !echo "Fixing nested models directory..."
                !mv {correct_models_path} {LOCAL_WORKSPACE}/models_local
                !mv {nested_models_path} {LOCAL_WORKSPACE}
                !rm -rf {LOCAL_WORKSPACE}/models_local
                !echo "Nested models directory fixed."


            USE_GIT_CLONE = False
        except tarfile.ReadError:
            print("Error reading the tar.gz file. It might be corrupted.")
            USE_GIT_CLONE = True
        finally:
            # Clean up the copied tar file
            if os.path.exists(LOCAL_TAR_PATH):
                os.remove(LOCAL_TAR_PATH)
    else:
        !echo "ComfyUI archive not found in Google Drive. Proceeding with git clone."
        USE_GIT_CLONE = True

    if USE_GIT_CLONE:
        !echo "Syncing files from Google Drive to local disk..."
        !mkdir -p {LOCAL_WORKSPACE}
        !git clone https://github.com/comfyanonymous/ComfyUI.git {LOCAL_WORKSPACE}
        !rsync -av --progress --exclude 'models/' --exclude '.git/' --exclude '__pycache__/' "{DRIVE_WORKSPACE}/" "{LOCAL_WORKSPACE}/"
        !ln -s "{DRIVE_WORKSPACE}/models" "{LOCAL_WORKSPACE}/models"

%cd {LOCAL_WORKSPACE}

if OPTIONS['UPDATE_COMFY_UI']:
  !echo -= Updating ComfyUI =-
  !git pull

!echo -= Install dependencies =-
!pip3 install accelerate
!pip3 install einops transformers>=4.28.1 safetensors>=0.4.2 aiohttp pyyaml Pillow scipy tqdm psutil tokenizers>=0.13.3
!pip3 install torchsde
!pip3 install kornia>=0.7.1 spandrel soundfile sentencepiece
!pip3 install -r requirements.txt

if OPTIONS['USE_COMFYUI_MANAGER']:
  %cd custom_nodes
  ![ ! -d ComfyUI-Manager ] && echo -= Initial setup ComfyUI-Manager =- && git clone https://github.com/ltdrdata/ComfyUI-Manager
  %cd ComfyUI-Manager
  !git pull
  %cd {LOCAL_WORKSPACE} # Return to local workspace after updating manager

if OPTIONS['INSTALL_CUSTOM_NODES_DEPENDENCIES']:
  !echo -= Install custom nodes dependencies =-
  !pip install GitPython
  !python custom_nodes/ComfyUI-Manager/cm-cli.py restore-dependencies

# Sync updated files back to Google Drive
if OPTIONS['USE_GOOGLE_DRIVE']:
    !echo "Syncing updated files from local disk to Google Drive..."
    !rsync -av --progress --exclude 'models/' --exclude '__pycache__/' "{LOCAL_WORKSPACE}/" "{DRIVE_WORKSPACE}/"

In [ ]:
import os
import yaml
import subprocess

def download_models(config_path):
    """
    Downloads models based on a configuration file.

    Args:
        config_path (str): The path to the YAML configuration file in Google Drive.
    """
    try:
        with open(config_path, 'r') as f:
            config = yaml.safe_load(f)
    except FileNotFoundError:
        print(f"Error: Config file not found at {config_path}")
        return

    if 'models' not in config:
        print("Error: 'models' key not found in the config file.")
        return

    for model in config['models']:
        if 'url' not in model or 'output' not in model:
            print(f"Skipping invalid model entry: {model}")
            continue

        url = model['url']
        output_path = model['output']
        output_dir = os.path.dirname(output_path)

        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

        if os.path.exists(output_path):
            print(f"Model already exists: {output_path}")
        else:
            print(f"Downloading model from {url} to {output_path}...")
            # Ensure the directory exists before downloading
            if not os.path.exists(output_dir):
                os.makedirs(output_dir)

            # Use subprocess to capture the output and return code of aria2c
            result = subprocess.run(
                ['aria2c', '--console-log-level=error', '-c', '-x', '16', '-s', '16', '-k', '1M', url, '-o', output_path],
                capture_output=True,
                text=True
            )

            print(result.stdout)
            print(result.stderr)

            # Check for successful download based on aria2c return code
            if result.returncode == 0:
                print("Download complete.")
            else:
                print("Download failed.")
                # Optional: Print error message if subprocess failed
                if result.returncode != 0:
                    print(f"aria2c command failed with return code {result.returncode}")
                elif "ERROR" in result.stderr:
                     print(f"aria2c reported an error: {result.stderr}")
                # Re-check if file exists after a non-zero return code to provide more info
                if not os.path.exists(output_path):
                     print(f"Output file not found at {output_path} after download attempt.")




In [ ]:
# Example usage (assuming your config file is in your Google Drive)
config_file = "/content/drive/MyDrive/ComfyUI/models/model_config.yaml"
download_models(config_file)

In [ ]:
import atexit, requests, subprocess, time, re, os
from random import randint
from threading import Timer
from queue import Queue
def cloudflared(port, metrics_port, output_queue):
    atexit.register(lambda p: p.terminate(), subprocess.Popen(['/content/cloudflared-linux-amd64', 'tunnel', '--url', f'http://127.0.0.1:{port}', '--metrics', f'127.0.0.1:{metrics_port}'], stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT))
    attempts, tunnel_url = 0, None
    while attempts < 10 and not tunnel_url:
        attempts += 1
        time.sleep(3)
        try:
            tunnel_url = re.search("(?P<url>https?:\/\/[^\s]+.trycloudflare.com)", requests.get(f'http://127.0.0.1:{metrics_port}/metrics').text).group("url")
        except:
            pass
    if not tunnel_url:
        raise Exception("Can't connect to Cloudflare Edge")
    output_queue.put(tunnel_url)
output_queue, metrics_port = Queue(), randint(8100, 9000)
thread = Timer(2, cloudflared, args=(8188, metrics_port, output_queue))
thread.start()
thread.join()
tunnel_url = output_queue.get()
os.environ['webui_url'] = tunnel_url
print(tunnel_url)

In [ ]:
%cd {LOCAL_WORKSPACE}
!python main.py --dont-print-server

In [ ]:
import os
import tarfile

current_dir = !pwd
LOCAL_WORKSPACE = f"{current_dir[0]}/ComfyUI_local"
LOCAL_TAR_PATH = f"{current_dir[0]}/ComfyUI_local.tar.gz"
DRIVE_TAR_PATH = "/content/drive/MyDrive/ComfyUI_local.tar.gz"

# Create the tar.gz archive
print(f"Creating archive from {LOCAL_WORKSPACE}...")
try:
    with tarfile.open(LOCAL_TAR_PATH, "w:gz") as tar:
        tar.add(LOCAL_WORKSPACE, arcname=os.path.basename(LOCAL_WORKSPACE))
    print(f"Archive created at {LOCAL_TAR_PATH}")

    # Copy the archive to Google Drive, overwriting if it exists
    print(f"Copying archive to Google Drive at {DRIVE_TAR_PATH}...")
    !cp "{LOCAL_TAR_PATH}" "{DRIVE_TAR_PATH}"
    print("Copy complete.")

    # Clean up the local tar file
    if os.path.exists(LOCAL_TAR_PATH):
        os.remove(LOCAL_TAR_PATH)
        print(f"Cleaned up local archive {LOCAL_TAR_PATH}")

except Exception as e:
    print(f"An error occurred: {e}")